Required data files

wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# print(spacex_df[['Launch Site']].groupby('Launch Site', as_index=False).sum())
site_list = {}
for site in spacex_df[['Launch Site']].groupby('Launch Site', as_index=False).sum().iterrows():
    site_list['site'+ str(site[0]+1)] = site[1].to_list()[0]
# print(site_list)

def site_data(site_name):
    location_name = site_list[site_name]
    site_loc_data = spacex_df[spacex_df['Launch Site'] == location_name]
    site_loc_data['classstr'] = 1
    return site_loc_data 
# print(site_data('site2').to_string())
def payload_fun(site_name,payload_slider):
    if site_name == 'ALL':
        return spacex_df[spacex_df['Payload Mass (kg)'].between(payload_slider[0], payload_slider[1])]
    else:
        payload_fun_data = site_data(site_name)
        return payload_fun_data[payload_fun_data['Payload Mass (kg)'].between(payload_slider[0], payload_slider[1])]
    
# print(payload_fun('site2',[10,9000]).to_string())
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', options=[{'label': 'All Sites', 'value': 'ALL'},{'label': site_list['site1'], 'value': 'site1'},{'label': site_list['site2'], 'value': 'site2'},{'label': site_list['site3'], 'value': 'site3'},{'label': site_list['site4'], 'value': 'site4'}], value='ALL', placeholder='Select a Launch Site here', searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
           # Output(component_id='success-payload-scatter-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'),)
               #  Input(component_id="payload-slider", component_property="value"))
           

def get_graph(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Successful Flights From All Launch Pads')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.pie(site_data(entered_site), values='classstr', names='class',  title='Total Success & Failure Launch From '+str(site_list[entered_site]))
        # fig = px.pie(pd.DataFrame({'class':['10','0']}), values='class', names='class',  title='title')
        return fig



@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value"))

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
def get_scatter_plot(entered_site,entered_payload):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(payload_fun(entered_site,entered_payload), x='Payload Mass (kg)', y='class', color="Booster Version Category", 
        title='Correlation b/w Sucess & All Landing Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.scatter(payload_fun(entered_site,entered_payload), x='Payload Mass (kg)', y='class', color="Booster Version Category", 
        title='Correlation b/w Sucess & '+str(site_list[entered_site]))
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
